In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 데이터 경로 변수

In [2]:
DATA_PATH = "/content/drive/MyDrive/data/"
DATA_PATH

'/content/drive/MyDrive/data/'

- 시드값

In [3]:
SEED = 42

- 데이터 불러오기

In [4]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터
submit_proba = pd.read_csv(f"{DATA_PATH}store_submission.csv")

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [5]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_3.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_3.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 435), (12225, 435))

# 결측치 처리

In [6]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0
3회이상count,4214
구매금액표준편차,388
구매금액왜도,840
구매금액첨도,1265


In [7]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0
3회이상count,3284
구매금액표준편차,242
구매금액왜도,599
구매금액첨도,954


In [8]:

train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)
train_ft["구매금액왜도"] = train_ft["구매금액왜도"].fillna(0)
train_ft["구매금액첨도"] = train_ft["구매금액첨도"].fillna(0)
train_ft["3회이상count"] = train_ft["3회이상count"].fillna(0)



test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)
test_ft["구매금액왜도"] = test_ft["구매금액왜도"].fillna(0)
test_ft["구매금액첨도"] = test_ft["구매금액첨도"].fillna(0)
test_ft["3회이상count"] = test_ft["3회이상count"].fillna(0)



In [9]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [10]:
train_ft['가족단위구매비율'] = (train_ft['대분류_아동_cnt'] + train_ft['유아관련count']) / train_ft['구매건수']
test_ft['가족단위구매비율'] = (test_ft['대분류_아동_cnt'] + test_ft['유아관련count']) / test_ft['구매건수']

In [11]:
train_ft.drop('유아관련count', axis=1, inplace=True)
test_ft.drop('유아관련count', axis=1, inplace=True)

#11-10 추가피처

In [12]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])
train_tr['구매일'] = train_tr['구매일시'].dt.date
test_tr['구매일'] = test_tr['구매일시'].dt.date

In [13]:
grouped = train_tr.groupby(['ID', '구매일'])['구매일시']
daily_stay_time = grouped.agg(['min', 'max'])
daily_stay_time['체류시간'] = (daily_stay_time['max'] - daily_stay_time['min']).dt.total_seconds()
average_stay_time = daily_stay_time.groupby('ID')['체류시간'].mean()
train_ft = train_ft.merge(average_stay_time, how='left',on="ID")
grouped = test_tr.groupby(['ID', '구매일'])['구매일시']
daily_stay_time = grouped.agg(['min', 'max'])
daily_stay_time['체류시간'] = (daily_stay_time['max'] - daily_stay_time['min']).dt.total_seconds()
average_stay_time = daily_stay_time.groupby('ID')['체류시간'].mean()
test_ft = test_ft.merge(average_stay_time, how='left',on="ID")
train_ft["구매당체류시간"] = (train_ft["체류시간"]/60)/train_ft["구매건수"]
test_ft["구매당체류시간"] = (test_ft["체류시간"]/60)/test_ft["구매건수"]

In [14]:
grouped = train_tr.groupby(['ID', '구매일','지점코드'])['구매일시']
daily_stay_time = grouped.agg(['min', 'max'])
daily_stay_time['체류시간'] = (daily_stay_time['max'] - daily_stay_time['min']).dt.total_seconds()
daily_stay_time = daily_stay_time.reset_index()
average_stay_time = daily_stay_time.groupby(['ID','지점코드'])['체류시간'].sum().reset_index()
train_tmp = pd.pivot_table(average_stay_time,index="ID",columns="지점코드", values = "체류시간",aggfunc="mean",fill_value=0).add_prefix("지점코드별체류시간_").reset_index()
train_ft = train_ft.merge(train_tmp,how="left",on="ID")


grouped = test_tr.groupby(['ID', '구매일','지점코드'])['구매일시']
daily_stay_time = grouped.agg(['min', 'max'])
daily_stay_time['체류시간'] = (daily_stay_time['max'] - daily_stay_time['min']).dt.total_seconds()
daily_stay_time = daily_stay_time.reset_index()
average_stay_time = daily_stay_time.groupby(['ID','지점코드'])['체류시간'].sum().reset_index()
test_tmp = pd.pivot_table(average_stay_time,index="ID",columns="지점코드", values = "체류시간",aggfunc="mean",fill_value=0).add_prefix("지점코드별체류시간_").reset_index()
for col in train_tmp.columns:
    if col not in test_tmp.columns:
        test_tmp[col] = 0

test_tmp = test_tmp[train_tmp.columns]
test_ft = test_ft.merge(test_tmp, how = "left", on = "ID")


# 특성 공학(Feature Engineering)

- ID 변수 제외

In [15]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 440), (12225, 440))

## Feature Encoding

In [16]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
최대구매액_대분류,28
최소구매액_대분류,28


In [17]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 381.7 kB/s eta 0:00:00


In [18]:
import category_encoders as ce

In [19]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점","최소구매액_대분류","최대구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점","최소구매액_대분류","최대구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 500), (12225, 500))

In [20]:
cols

['주구매지점', '최대구매액_대분류', '최소구매액_대분류']

In [21]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 497), (12225, 497))

In [22]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [23]:
train_ft.head()

,구매횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,최대구매액_대분류_19,최대구매액_대분류_20,최대구매액_대분류_21,최대구매액_대분류_22,최대구매액_대분류_23,최대구매액_대분류_24,최대구매액_대분류_25,최대구매액_대분류_26,최대구매액_대분류_27,최대구매액_대분류_28
0,16,11,23,0.050000,0.250000,0.400000,0.300000,3,0.300000,0.450000,...,0,0,0,0,0,0,0,0,0,0
1,41,21,16,0.357143,0.166667,0.357143,0.119048,3,0.214286,0.476190,...,0,0,0,0,0,0,0,0,0,0
2,102,56,6,0.464912,0.140351,0.175439,0.219298,0,0.122807,0.728070,...,0,0,0,0,0,0,0,0,0,0
3,191,92,3,0.379147,0.180095,0.236967,0.203791,3,0.161137,0.616114,...,0,0,0,0,0,0,0,0,0,0
4,55,27,11,0.112903,0.612903,0.209677,0.064516,4,0.096774,0.741935,...,0,0,0,0,0,0,0,0,0,0


## Feature Scaling

In [24]:
np.isinf(train_ft).sum().sum(), np.isinf(test_ft).sum().sum()

(0, 0)

In [25]:
train_ft.replace([np.inf, -np.inf], 0, inplace=True)
test_ft.replace([np.inf, -np.inf], 0, inplace=True)

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [27]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,구매횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,최대구매액_대분류_19,최대구매액_대분류_20,최대구매액_대분류_21,최대구매액_대분류_22,최대구매액_대분류_23,최대구매액_대분류_24,최대구매액_대분류_25,최대구매액_대분류_26,최대구매액_대분류_27,최대구매액_대분류_28
0,-0.437126,-0.369867,0.002987,-1.029777,0.001191,0.838272,0.338186,0.109631,0.718557,-0.654150,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
1,0.239394,0.144110,-0.356452,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.269465,-0.542415,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
2,1.890101,1.943028,-0.869935,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,-0.209830,0.532172,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
3,4.298510,3.793345,-1.023980,0.420933,-0.327474,0.008592,-0.135636,0.109631,-0.009001,0.054536,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
4,0.618244,0.452496,-0.613193,-0.752532,1.707410,-0.130285,-0.821561,0.693965,-0.346226,0.591326,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497


# 정답 데이터

In [28]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# cv 점수 확인해보기

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

# 피처셀렉션

In [30]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_ft, target, test_size=0.2, random_state=42)

model = LGBMClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

feature_importances = model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': train_ft.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

feature_importance_df.head(50)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4687, number of negative: 7265
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.344141 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 52386
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 472
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.392152 -> initscore=-0.438276
[LightGBM] [Info] Start training from score -0.438276


,Feature,Importance
430,가족단위구매비율,64
4,여름_구매비율,54
14,방문평균거래평균횟수,52
10,18시이후구매비율,49
411,요일별_구매횟수비율_3,46
32,대분류_생식품_cnt,40
2,구매주기,39
187,중_pivot_횟수비율_수입종합화장품,38
37,최소구매액,37
233,중_pivot_횟수비율_용기보증,36


In [31]:
feature_importance_df[feature_importance_df["Importance"]==0]

,Feature,Importance
488,최대구매액_대분류_20,0
431,체류시간,0
448,최소구매액_대분류_8,0
487,최대구매액_대분류_19,0
446,최소구매액_대분류_6,0
...,...,...
274,중_pivot_횟수비율_종합_수입,0
278,중_pivot_횟수비율_즉석조리,0
279,중_pivot_횟수비율_지갑_벨트,0
280,중_pivot_횟수비율_직수입침구,0


In [32]:
rows_to_drop = feature_importance_df[feature_importance_df["Importance"]==0].index
feature_importance_df.drop(rows_to_drop, inplace=True)

In [33]:
cols = feature_importance_df['Feature'].tolist()
train_ft = train_ft[cols]
test_ft = test_ft[cols]
train_ft.shape, test_ft.shape

((14940, 321), (12225, 321))

#테스트

In [38]:
%pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 3.9 MB/s eta 0:00:00


In [39]:
%pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 5.9 MB/s eta 0:00:00


In [40]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.3 MB/s eta 0:00:00


In [41]:
from flaml import AutoML
auto_ml = AutoML()
params = {
    "metric": "macro_f1",
    "estimator_list":['lgbm','xgboost','histgb','catboost'],
    "task" : "classification",
    "time_budget": 60*60,
    "seed" : 42,
    "early_stop": True
}

auto_ml.fit(train_ft, target, **params)

[flaml.automl.logger: 11-15 02:33:27] {1728} INFO - task = classification
[flaml.automl.logger: 11-15 02:33:27] {1739} INFO - Evaluation method: holdout
[flaml.automl.logger: 11-15 02:33:27] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-15 02:33:27] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'histgb', 'catboost']
[flaml.automl.logger: 11-15 02:33:27] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 11-15 02:33:29] {2393} INFO - Estimated sufficient time budget=20702s. Estimated necessary time budget=39s.
[flaml.automl.logger: 11-15 02:33:29] {2442} INFO -  at 4.4s,	estimator lgbm's best error=0.6223,	best estimator lgbm's best error=0.6223
[flaml.automl.logger: 11-15 02:33:30] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-15 02:33:33] {2442} INFO -  at 7.5s,	estimator lgbm's best error=0.6223,	best estimator lgbm's best error=0.6223
[flaml.automl.logger: 11-15 02:33:33] {2258} INFO - 

In [42]:
1-auto_ml.best_loss

0.7206982064891602

In [ ]:
pred = auto_ml.predict(test_ft)
pred

In [ ]:
pred_proba = auto_ml.predict_proba(test_ft)[:,1]
pred_proba

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [ ]:
submit

In [ ]:
submit["target"] = pred
submit

- 예측 결과를 csv 파일로 저장하여 제출

In [ ]:
submit.to_csv(f"{DATA_PATH}submit.csv",index=False)

In [ ]:
submit_proba["target"]=pred_proba
submit_proba

In [ ]:
submit_proba.to_csv(f"{DATA_PATH}submit_proba.csv",index=False)